# Notebook's objective

# Libraries

In [1]:
# Use this initial code to work in the notebook as if it were a module, that 
# is, to be able to export classes and functions from other subpackages.

import os
import sys

package_path = os.path.abspath('.').split(os.sep + 'notebooks')[0]
if package_path not in sys.path:
    sys.path.append(package_path)

%load_ext autoreload
%autoreload 2

# Load data

# Load LLM

In [5]:
'''This following code will set the CURL_CA_BUNDLE environment variable to an empty string in the Python os module'''

import os
os.environ['CURL_CA_BUNDLE'] = ''

In [7]:
import requests
from huggingface_hub import configure_http_backend

def backend_factory() -> requests.Session:
    session = requests.Session()
    session.verify = False
    return session

configure_http_backend(backend_factory=backend_factory)

# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("unsloth/Llama-3.2-1B-Instruct")
model = AutoModelForCausalLM.from_pretrained("unsloth/Llama-3.2-1B-Instruct")

c:\Users\rgs1jmo\Documents\risk_analytics\tuyabot-llm\.venv\lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\rgs1jmo\Documents\risk_analytics\tuyabot-llm\.venv\lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\rgs1jmo\Documents\risk_analytics\tuyabot-llm\.venv\lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\rgs1jmo\.cache\huggingface\hub

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.llms import HuggingFacePipeline
model_id = 'unsloth/Llama-3.2-1B-Instruct'

import requests
from huggingface_hub import configure_http_backend

def backend_factory() -> requests.Session:
    session = requests.Session()
    session.verify = False
    return session

configure_http_backend(backend_factory=backend_factory)

# #descarga de componentes base del modelo
# # tokenizer = AutoTokenizer.from_pretrained(model_id)
# # model = AutoModelForCausalLM.from_pretrained(model_id)

tokenizer = AutoTokenizer.from_pretrained(model_id, use_auth_token=True, trust_remote_code=True, verify=False)
model = AutoModelForCausalLM.from_pretrained(model_id, use_auth_token=True, trust_remote_code=True, verify=False)

#creacion del pipeline del modelo
pipe = pipeline(
            "text-generation",
            model=model, 
            tokenizer=tokenizer, 
            max_length=2000, 
            # device = "cuda"
 )

#conversión a uso api tipo langchain
local_llm = HuggingFacePipeline(pipeline=pipe)

Device set to use cpu
C:\Users\rgs1jmo\AppData\Local\Temp\ipykernel_12224\3023481758.py:22: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  local_llm = HuggingFacePipeline(pipeline=pipe)


In [11]:
# Función para generar texto con el modelo
def generate_text(prompt):
    response = local_llm(prompt)
    return response

# Prueba del modelo
prompt = "Hola, ¿cómo estás?"
generated_text = generate_text(prompt)
# print("Texto generado:", generated_text)


KeyboardInterrupt: 